In [1]:
import requests
# url = 'https://discuss.streamlit.io/t/build-a-multi-agent-ai-researcher-using-ollama-langgraph-and-streamlit/116726'
url = 'https://en.wikipedia.org/wiki/LangChain'
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:147.0) Gecko/20100101 Firefox/147.0'
}
response = requests.get(url=url, headers=headers)

page_content = response.content
print(page_content)

b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>LangChain - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page_content, 'html.parser')

hrefs = [a['href'] for a in soup.find_all('a', href=True) if a['href'].startswith('http')]
title = soup.title.string

print(title)
print(len(hrefs),hrefs)

LangChain - Wikipedia
66 ['https://donate.wikimedia.org/?wmf_source=donate&wmf_medium=sidebar&wmf_campaign=en.wikipedia.org&uselang=en', 'https://donate.wikimedia.org/?wmf_source=donate&wmf_medium=sidebar&wmf_campaign=en.wikipedia.org&uselang=en', 'https://ca.wikipedia.org/wiki/LangChain', 'https://fa.wikipedia.org/wiki/%D9%84%D9%86%DA%AF_%DA%86%DB%8C%D9%86', 'https://fr.wikipedia.org/wiki/LangChain', 'https://ko.wikipedia.org/wiki/%EB%9E%AD%EC%B2%B4%EC%9D%B8', 'https://hi.wikipedia.org/wiki/%E0%A4%B2%E0%A5%88%E0%A4%82%E0%A4%97%E0%A4%9A%E0%A5%87%E0%A4%A8', 'https://ja.wikipedia.org/wiki/LangChain', 'https://pt.wikipedia.org/wiki/LangChain', 'https://ru.wikipedia.org/wiki/LangChain', 'https://simple.wikipedia.org/wiki/LangChain', 'https://th.wikipedia.org/wiki/%E0%B9%81%E0%B8%A5%E0%B8%87%E0%B9%80%E0%B8%8A%E0%B8%99', 'https://tr.wikipedia.org/wiki/LangChain', 'https://uk.wikipedia.org/wiki/LangChain', 'https://zh.wikipedia.org/wiki/LangChain', 'https://www.wikidata.org/wiki/Special:Entit

In [3]:
from urllib.parse import urlparse

def get_root_host(url):
    parsed_url = urlparse(url)
    hostname = parsed_url.hostname
    parts = hostname.split('.')
    # Return the last two parts joined by a dot
    return '.'.join(parts[-2:])

result = get_root_host(response.url)
print(result)  

wikipedia.org


In [4]:
links = {'internal': [], 'external': []}
internal_host = get_root_host(response.url) 

for url in hrefs:
    if get_root_host(url) == internal_host:
        links['internal'].append(url)
    else:
        links['external'].append(url)

print(links)

{'internal': ['https://ca.wikipedia.org/wiki/LangChain', 'https://fa.wikipedia.org/wiki/%D9%84%D9%86%DA%AF_%DA%86%DB%8C%D9%86', 'https://fr.wikipedia.org/wiki/LangChain', 'https://ko.wikipedia.org/wiki/%EB%9E%AD%EC%B2%B4%EC%9D%B8', 'https://hi.wikipedia.org/wiki/%E0%A4%B2%E0%A5%88%E0%A4%82%E0%A4%97%E0%A4%9A%E0%A5%87%E0%A4%A8', 'https://ja.wikipedia.org/wiki/LangChain', 'https://pt.wikipedia.org/wiki/LangChain', 'https://ru.wikipedia.org/wiki/LangChain', 'https://simple.wikipedia.org/wiki/LangChain', 'https://th.wikipedia.org/wiki/%E0%B9%81%E0%B8%A5%E0%B8%87%E0%B9%80%E0%B8%8A%E0%B8%99', 'https://tr.wikipedia.org/wiki/LangChain', 'https://uk.wikipedia.org/wiki/LangChain', 'https://zh.wikipedia.org/wiki/LangChain', 'https://en.wikipedia.org/w/index.php?title=LangChain&oldid=1332531370'], 'external': ['https://donate.wikimedia.org/?wmf_source=donate&wmf_medium=sidebar&wmf_campaign=en.wikipedia.org&uselang=en', 'https://donate.wikimedia.org/?wmf_source=donate&wmf_medium=sidebar&wmf_campaign

In [5]:
text = soup.get_text()
# text = re.sub(r'\s+', ' ', text).strip()
text = [line.strip() for line in text.split('\n') if len(line.strip().split()) >= 5]
text = '\n'.join(text)

print(text)

Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us
HelpLearn to editCommunity portalRecent changesUpload fileSpecial pages
Donate Create account Log in
Toggle the table of contents
What links hereRelated changesUpload filePermanent linkPage informationCite this pageGet shortened URLDownload QR code
From Wikipedia, the free encyclopedia
Language model application development framework
LangChainDeveloperHarrison ChaseInitial releaseOctober 2022Stable release0.1.16[1]
/ 11 April 2024; 21 months ago (11 April 2024)
Repositorygithub.com/langchain-ai/langchainWritten inPython and JavaScriptTypeSoftware framework for large language model application developmentLicenseMIT LicenseWebsiteLangChain.com
Free and open-source software portal
LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in gener

In [6]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse


def scrap_webpage(url, headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:147.0) Gecko/20100101 Firefox/147.0'}, include_links=True, timeout=10, content_line_words = 5):
    response = requests.get(url, headers=headers, timeout=timeout)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')
    document = {'title': soup.title.string, 'url': response.url }

    def get_root_host(url):
        parsed_url = urlparse(url)
        hostname = parsed_url.hostname
        parts = hostname.split('.')
        # Return the last two parts joined by a dot
        return '.'.join(parts[-2:])

    internal_host = get_root_host(response.url) 

    if include_links:
        hrefs = [a['href'] for a in soup.find_all('a', href=True) if a['href'].startswith('http')]
        links = {'internal': [], 'external': []}
        for url in hrefs:
            if get_root_host(url) == internal_host:
                links['internal'].append(url)
            else:
                links['external'].append(url)
        document['links'] = links

    text = soup.get_text()
    # text = re.sub(r'\s+', ' ', text).strip()
    text = [line.strip() for line in text.split('\n') if len(line.strip().split()) >= content_line_words]
    document['content'] = '\n'.join(text)

    return document
        
scrap_webpage('https://en.wikipedia.org/wiki/LangChain')

{'title': 'LangChain - Wikipedia',
 'url': 'https://en.wikipedia.org/wiki/LangChain',
 'links': {'internal': ['https://ca.wikipedia.org/wiki/LangChain',
   'https://fa.wikipedia.org/wiki/%D9%84%D9%86%DA%AF_%DA%86%DB%8C%D9%86',
   'https://fr.wikipedia.org/wiki/LangChain',
   'https://ko.wikipedia.org/wiki/%EB%9E%AD%EC%B2%B4%EC%9D%B8',
   'https://hi.wikipedia.org/wiki/%E0%A4%B2%E0%A5%88%E0%A4%82%E0%A4%97%E0%A4%9A%E0%A5%87%E0%A4%A8',
   'https://ja.wikipedia.org/wiki/LangChain',
   'https://pt.wikipedia.org/wiki/LangChain',
   'https://ru.wikipedia.org/wiki/LangChain',
   'https://simple.wikipedia.org/wiki/LangChain',
   'https://th.wikipedia.org/wiki/%E0%B9%81%E0%B8%A5%E0%B8%87%E0%B9%80%E0%B8%8A%E0%B8%99',
   'https://tr.wikipedia.org/wiki/LangChain',
   'https://uk.wikipedia.org/wiki/LangChain',
   'https://zh.wikipedia.org/wiki/LangChain',
   'https://en.wikipedia.org/w/index.php?title=LangChain&oldid=1332531370'],
  'external': ['https://donate.wikimedia.org/?wmf_source=donate&wmf_m

In [16]:
from langchain_core.tools import tool
from typing import Annotated, List, Dict, Any, Optional

@tool
def web_scrape(url: Annotated[str, "The URL to scrape"]) -> Annotated[ Dict[str, Any], "The scraped webpage data"]:
    """Navigates to a webpage and scrapes its title and content."""
    return scrap_webpage(url, include_links=False)

# @tool
# def deep_scrape(url: Annotated[str, "The URL to scrape"]) -> Annotated[ Dict[str, Any], "The scraped webpage data"]:
#     """Navigates to a webpage and scrapes its title, content, and links."""
#     return scrap_webpage(url, include_links=True)

web_scrape.invoke(input={"url": "https://en.wikipedia.org/wiki/LangChain"})

{'title': 'LangChain - Wikipedia',
 'url': 'https://en.wikipedia.org/wiki/LangChain',
 'content': 'Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\nDonate Create account Log in\nToggle the table of contents\nWhat links hereRelated changesUpload filePermanent linkPage informationCite this pageGet shortened URLDownload QR code\nFrom Wikipedia, the free encyclopedia\nLanguage model application development framework\nLangChainDeveloperHarrison ChaseInitial releaseOctober 2022Stable release0.1.16[1]\n/ 11 April 2024; 21 months ago\xa0(11 April 2024)\nRepositorygithub.com/langchain-ai/langchainWritten inPython and JavaScriptTypeSoftware framework for large language model application developmentLicenseMIT LicenseWebsiteLangChain.com\nFree and open-source software portal\nLangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a l

In [17]:
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from typing import TypedDict

from langchain_ollama import ChatOllama
from langfuse.langchain import CallbackHandler

llm_model = ChatOllama(model="qwen3")

# Initialize Langfuse CallbackHandler for LangGraph/Langchain (tracing)
langfuse_handler = CallbackHandler() 
llm_config = {"configurable": {"thread_id": "internet_agent"}, "recursion_limit": 20, "callbacks": [langfuse_handler]}

internet_agent_prompt = """You are a helpful internet researcher. 
    - Navigate to a webpage and scrape its content without the links.
    - Be concise, and provide a 200 summary of the webpage content.
    - Tag the webpage content as [user post, news article, documentation, sales page, product advertisement, book, etc.]
    - Determine it it contains relevant information for the topic.
    - do not use bullets or numbered lists.
    - Provide your answer in markdown format.
"""
# Define structured output sche
class SummaryOutput(TypedDict):
    classification: str
    is_relevant_information: bool
    title: str
    summary: str

# Define the tool strategy
# Create the agent
internet_agent = create_agent(name="internet_agent", model=llm_model, tools=[web_scrape], 
    # system_prompt=internet_agent_prompt, 
    # response_format=ToolStrategy(List)
    )

messages = {"messages": [("user", """extrae una lista de palabras clave que debería tener un currículum para esta oferta de trabajo, unicamente las palabras. 
    url:https://www.linkedin.com/jobs/view/4362181901/
    Example output format: ['Python', 'Data Science']""")]}
async for step in internet_agent.astream(messages, config=llm_config, stream_mode="values" ):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

extrae una lista de palabras clave que debería tener un currículum para esta oferta de trabajo, unicamente las palabras. 
    url:https://www.linkedin.com/jobs/view/4362181901/
    Example output format: ['Python', 'Data Science']
================================== Ai Message ==================================
Name: internet_agent

<think>
Okay, the user wants a list of keywords for a resume based on a job posting. The URL provided is a LinkedIn job listing. First, I need to scrape the page to get the content. The function provided is web_scrape, which takes a URL and returns the title and content. 

So, I'll call the web_scrape function with the given URL. Once I have the content, I'll need to extract the keywords. But wait, the user just wants the keywords, not the entire content. Maybe the job description has specific terms like required skills, tools, or experience.

After scraping, I should look for 

In [38]:
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from typing import TypedDict

from langchain_ollama import ChatOllama
from langfuse.langchain import CallbackHandler

def summarize_text_llm(text: str, 
    llm_model = ChatOllama(model="qwen3"),
    llm_config = {"configurable": {"thread_id": "summarization_agent"}, "recursion_limit": 2, "callbacks": [CallbackHandler()]},
    system_prompt = "Summarize the given text in 100 words /no_think",
    ) -> str :

    summarization_agent = create_agent(name="internet_agent", model=llm_model, system_prompt=system_prompt )
    return summarization_agent.invoke({"messages": [("user", text)]}, config=llm_config)


In [105]:
# 1. Scrap webpage with links
# 2. Scrap each link and extract text
# 3. Sumarize the extracted text, extract the topic and keywords from it
# 4. Store relevant text in a database, delete the rest


url = 'https://www.linkedin.com/jobs/view/4362181901/'
# url = 'https://en.wikipedia.org/wiki/LangChain'
# 1. Scrap webpage with links

webpage = scrap_webpage(url = url, include_links=True)
print(webpage['title'])


DXC Technology hiring Analista técnico/funcional informático/a in Sant Cugat del Vallès, Catalonia, Spain | LinkedIn


In [106]:

summary = summarize_text_llm(text=webpage['content'], llm_model = ChatOllama(model="qwen3"), system_prompt = "Summarize the given text in 500 words /no_think",)
webpage['summary'] = summary['messages'][1].content

import textwrap

print(textwrap.fill(webpage['summary'], width=100))


<think>  </think>  DXC Technology is seeking a **Técnico/Funcional Informático/a** in **Sant Cugat
del Vallès, Catalonia, Spain**. This role is part of a service focused on the development and
maintenance of an application for managing public tenders in the Catalan public administration. The
position requires a high-level functional analyst with expertise in analyzing requirements,
designing IT solutions, and identifying inconsistencies in requirements. The candidate will
collaborate with multidisciplinary teams to draft and review technical and commercial proposals for
tenders, ensuring their accuracy and compliance with requirements.  Key responsibilities include
developing and maintaining standardized templates and documentation to streamline the tender
proposal process, participating in the definition of strategies for tenders, and analyzing past
tenders to improve future proposals. The role also involves working with tools for enterprise
architecture modeling, such as Archi, Visio

In [108]:

topic = summarize_text_llm(text=webpage['summary'], llm_model = ChatOllama(model="qwen3:1.7b"), system_prompt = "Extract the topic of the given text in no more than 10 words /no_think",)
webpage['topic'] = topic['messages'][1].content

print(webpage['topic'])


DXC Technology hires Functional Analyst for public tenders in Catalonia.


In [120]:

keywords = summarize_text_llm(text=webpage['summary'], llm_model = ChatOllama(model="qwen3:1.7b", temperature=0.1), system_prompt = "Extract the list of keywords of the given text. separed by |. do not group them. do not include categories. /no_think",)
webpage['keywords'] = keywords['messages'][1].content

print(textwrap.fill(webpage['keywords'], width=100))


Técnico/Funcional Informático/a|Sant Cugat del Vallès|Catalonia|Spain|DXC Technology|public
tenders|development of applications|analysis of requirements|designing IT solutions|collaboration
with teams|developing templates|participating in strategies|analyzing past
tenders|Archi|Visio|UML|Low Code platforms|drafting technical and commercial proposals|regional IT
systems|project management methodologies|Microsoft tools|AI tools|Spanish|Catalan|professional
development|high-impact projects|supportive work environment|learning resources|transparent
recruitment process|fraudulent practices|no employment through social media|global leader|over 70
countries|8,000 professionals|cloud computing|AI|automation|IT services|digital
transformation|technological innovation


In [110]:
document_classes = [ "Article", "Advertise", "Blog Post", "Job Offer", "News", "Research Paper", 
    "Thesis", "Email", "Legal Document", "Instruction Manual", "Social Media Post", 
    "Product Review", "Service Review", "Technical Report", "User Guide", "White Paper", 
    "Script", "Press Release", "FAQ", "Resume/CV", "Invoice", "Newsletter", "Story", 
    "Code Documentation", "Policy Document", ]

document_type = summarize_text_llm(text=webpage['summary'], llm_model = ChatOllama(model="qwen3:1.7b"), system_prompt = f"Point wich type of document is the text ({','.join(document_classes)}). Do not elavorate. Do not format. /no_think",)
webpage['document_type'] = document_type['messages'][1].content

print(webpage['document_type'])


Job Offer


In [ ]:
display(webpage)

{'title': 'DXC Technology hiring Analista técnico/funcional informático/a in Sant Cugat del Vallès, Catalonia, Spain | LinkedIn',
 'url': 'https://www.linkedin.com/jobs/view/4362181901/',
 'links': {'internal': ['https://www.linkedin.com/legal/cookie-policy',
   'https://www.linkedin.com/mypreferences/g/guest-cookies',
   'https://www.linkedin.com/login?emailAddress=&fromSignIn=&fromSignIn=true&session_redirect=https%3A%2F%2Fwww.linkedin.com%2Fjobs%2Fview%2F4362181901%2F&trk=public_jobs_nav-header-signin',
   'https://www.linkedin.com/signup/cold-join?source=jobs_registration&session_redirect=https%3A%2F%2Fwww.linkedin.com%2Fjobs%2Fview%2F4362181901%2F&trk=public_jobs_nav-header-join',
   'https://www.linkedin.com/login?emailAddress=&fromSignIn=&fromSignIn=true&session_redirect=https%3A%2F%2Fwww.linkedin.com%2Fjobs%2Fview%2F4362181901%2F&trk=public_jobs_nav-header-signin',
   'https://www.linkedin.com/company/dxctechnology?trk=public_jobs_sub-nav-cta-optional-url',
   'https://www.link